# QFT Deployment on Neutral-Atom Systems
Over the course of the following tutorial, we will be implementing and deploying the Quantum Fourier Transform Circuit on the Bloqade framework as well as exploring neutral-atom specific optimizations that would take advantage of the parallelization provided by the platfrom.


The following two blocks are imports that set up both the mathematical foundations and the circuit representation tools that are used for optimizing thw Quantum Fourier Transform

In [1]:
import math

from bloqade import qasm2
from kirin.dialects import ilist

In [2]:
from bloqade.qasm2.emit import QASM2 # the QASM2 target
from bloqade.qasm2.parse import pprint # the QASM2 pretty printer

This block defines a custom compiler pass that rewrites a quantum circuit into the native neutral‑atom gate set using RydbergGateSetRewriteRule combined with a circuit walker. It then applies a folding pass (QASM2Fold) to unfold loops and reveal parallelization opportunities. If enabled, the pass applies UOpToParallel to group independent operations into parallel layers, reducing the circuit depth. In essence, this compiler pass first lowers and rewrites the circuit and then restructures it to exploit parallelism, similar to how GHZ circuits group entangling operations.

In [3]:
from bloqade.qasm2.rewrite.native_gates import RydbergGateSetRewriteRule
from kirin import ir
from kirin.rewrite import Walk
from bloqade.qasm2.passes import UOpToParallel, QASM2Fold


@ir.dialect_group(qasm2.extended)
def extended_opt(self):
    native_rewrite = Walk(RydbergGateSetRewriteRule(self)) # use Kirin's functionality to walk code line by line while applying neutral-atom gate decomposition as defined in Bloqade
    parallelize_pass = UOpToParallel(self) # review the code and apply parallelization using a heuristic
    agg_fold = QASM2Fold(self) # supports parallelization by unfolding loops to search for parallelization opportunities

    # here we define our new compiler pass
    def run_pass(
        kernel: ir.Method,
        *,
        fold: bool = True,
        typeinfer: bool = True,
        parallelize: bool = False,
    ):
        assert qasm2.extended.run_pass is not None
        qasm2.extended.run_pass(kernel, fold=fold, typeinfer=typeinfer) # apply the original run_pass to the lowered kernel
        native_rewrite.rewrite(kernel.code) # decompose all gates in the circuit to neutral atom gate set

        # here goes our parallelization optimizer; the order of the commands here matters!
        if parallelize:
            agg_fold.fixpoint(kernel)
            parallelize_pass(kernel)

    return run_pass

This block defines a function that constructs a log-depth GHZ state preparation circuit. It first computes the number of qubits from the exponent n and defines a helper function (layer_of_cx) that applies controlled-X gates with barriers in a structured pattern. The main nested function (ghz_log_depth_program) starts with a Hadamard gate on the first qubit, then iteratively applies these CX layers to build the entanglement, and finally performs measurements on all qubits. Both nested functions are optimized using the extended_opt decorator, which includes gate rewriting and parallelization to reduce the circuit depth.

In [4]:
def ghz_log_depth(qreg: qasm2.QReg, creg : qasm2.CReg, n: int, parallelize: bool = True):
    n_qubits = int(2**n)

    @extended_opt
    def layer_of_cx(i_layer: int, qreg: qasm2.QReg):
        step = n_qubits // (2**i_layer)
        for j in range(0, n_qubits, step):
            qasm2.cx(ctrl=qreg[j], qarg=qreg[j + step // 2])
            qasm2.barrier((qreg[j], qreg[j + step // 2]))


    @extended_opt(parallelize=parallelize)
    def ghz_log_depth_program():
        qasm2.h(qreg[0])
        for i in range(n):
            layer_of_cx(i_layer=i, qreg=qreg)
            
        for i in range(n_qubits):
            qasm2.measure(qreg[i],creg[i])
            
        return creg # return register for simulation

    return ghz_log_depth_program

This block defines a function to construct a Quantum Fourier Transform (QFT) circuit, optimized for parallel execution. It first computes the total number of qubits and creates a helper function (layer_of_cx_ghz) that applies controlled-X gates with barriers in a patterned manner using an offset and step parameter. The main nested function (qft_program) then builds the circuit by applying Hadamard gates, a series of Rz rotations, and multiple layers of these controlled operations in a structured loop, before measuring each qubit. Finally, the circuit is emitted using the QASM2 target and printed in a readable format, showcasing the parallelization enhancements applied via the extended_opt decorator.

The first section of the QFT is as follows, however, it is not in the native gate set for neutral-atom computers. Thus we decomposed the circuit into gates which are native to the architecture, and CNOTs that ressemble the GHZ problem, which shown in the tutorial to be written in the native gate set.

<div align="center">
<picture>
   <img src="ImageYQuantumHBlock.png" style="width: 35vw; min-width: 330px;" >
</picture>
</div>

<div align="center">
<picture>
   <img src="ImageYQuantumHBlockSecondExpantion.png" >
</picture>
</div>

<div align="center">
<picture>
   <img src="Untitled-1.png" >
</picture>
</div>

<div align="center">
<picture>
   <img src="Untitled-2.png" >
</picture>
</div>

These decompositions are from [nature](https://www.nature.com/articles/s41598-023-35625-3#Fig5) and show the CNOTs ressembling the GHZ allowing us to take advantage of the parallelization proposed in the GHZ tutorial provided.

<div align="center">
<picture>
   <img src="GHZ_linear.png" style="width: 35vw; min-width: 330px;" >
</picture>
</div>

<div align="center">
<picture>
   <img src="GHZ_parallel.png" style="width: 25vw; height: 25vw;" >
</picture>
</div>

In [5]:
import numpy as np

def qft(n: int, parallelize: bool = True):
    n_qubits = int(2**n)

    @qasm2.extended
    def layer_of_cx_ghz_series(start : int, n_qubits : int, qreg: qasm2.QReg):
        for i in range(start, n_qubits):
            qasm2.cx(qreg[0], qreg[i])

    @extended_opt
    def layer_of_cx(i_layer: int, qreg: qasm2.QReg):
        step = n_qubits // (2**i_layer)
        for j in range(0, n_qubits, step):
            qasm2.cx(ctrl=qreg[j], qarg=qreg[j + step // 2])
            qasm2.barrier((qreg[j], qreg[j + step // 2]))



    @extended_opt(parallelize=parallelize)
    def qft_program():
        qreg = qasm2.qreg(n_qubits)
        creg = qasm2.creg(n_qubits)
        
        for i in range(n_qubits):
            qasm2.h(qreg[i])
            sub_n = np.floor(np.log2(n_qubits - i))
            sub_length = np.int(2 ** np.floor(np.log2(n_qubits - i)))
            # Apply rotations in the middle
            for j in range(n - i):
                qasm2.rz(qreg[j + i], math.pi / (2**(j + 2)))

            for j in range(sub_n):
                layer_of_cx(i_layer=j, qreg=qreg)
            layer_of_cx_ghz_series(start=sub_length + 1, n_qubits=n_qubits, qreg=qreg)
            
            # Apply rotations in the middle
            for j in range(n - i):
                qasm2.rz(qreg[j + i], -math.pi / (2**(j + 2)))
            
            for j in range(sub_n):
                layer_of_cx(i_layer=j, qreg=qreg)
            layer_of_cx_ghz_series(start=sub_length + 1, n_qubits=n_qubits, qreg=qreg)
            qasm2.rz(qreg[i], ((2**(n - i + 1) - 1) * math.pi) / (2**(n - i + 2)))
            
        for i in range(n_qubits):
            qasm2.measure(qreg[i],creg[i])
            
        return creg # return register for simulation

    return qft_program

target = QASM2()
ast = target.emit(qft(3))
pprint(ast)

/Users/nachoblancasrodriguez/anaconda3/envs/yquantum-quera/lib/python3.13/site-packages/cirq/circuits/circuit_operation.py:173: FutureWarning: In cirq 1.6 the default value of `use_repetition_ids` will change to
`use_repetition_ids=False`. To make this warning go away, please pass
explicit `use_repetition_ids`, e.g., to preserve current behavior, use

  CircuitOperations(..., use_repetition_ids=True)
  warnings.warn(msg, FutureWarning)
/Users/nachoblancasrodriguez/anaconda3/envs/yquantum-quera/lib/python3.13/site-packages/cirq/circuits/circuit_operation.py:173: FutureWarning: In cirq 1.6 the default value of `use_repetition_ids` will change to
`use_repetition_ids=False`. To make this warning go away, please pass
explicit `use_repetition_ids`, e.g., to preserve current behavior, use

  CircuitOperations(..., use_repetition_ids=True)
  warnings.warn(msg, FutureWarning)
/Users/nachoblancasrodriguez/anaconda3/envs/yquantum-quera/lib/python3.13/site-packages/cirq/circuits/circuit_operation.

VerificationError: for loop body must have the same number of results as initializers

This block runs QFT on a given number of 2^n bits. It outputs each of the possible output states and the number of times each state was measured. Since the phase has no impact on the probability of a state being measured, the empirical probability distribution should read close to a uniform distribution across all states.

In [ ]:
from bloqade.pyqrack import PyQrack
from collections import Counter

device = PyQrack(dynamic_qubits=True, pyqrack_options={"isBinaryDecisionTree": False})
kernel = qft(1)
results = device.multi_run(kernel, _shots=2000)

def to_bitstrings(results):
    return Counter(map(lambda result:"".join(map(str, result)), results))

counts = to_bitstrings(results)

for key, value in counts.items():
    print(key, value)

/Users/nachoblancasrodriguez/anaconda3/envs/yquantum-quera/lib/python3.13/site-packages/cirq/circuits/circuit_operation.py:173: FutureWarning: In cirq 1.6 the default value of `use_repetition_ids` will change to
`use_repetition_ids=False`. To make this warning go away, please pass
explicit `use_repetition_ids`, e.g., to preserve current behavior, use

  CircuitOperations(..., use_repetition_ids=True)
  warnings.warn(msg, FutureWarning)
/Users/nachoblancasrodriguez/anaconda3/envs/yquantum-quera/lib/python3.13/site-packages/cirq/circuits/circuit_operation.py:173: FutureWarning: In cirq 1.6 the default value of `use_repetition_ids` will change to
`use_repetition_ids=False`. To make this warning go away, please pass
explicit `use_repetition_ids`, e.g., to preserve current behavior, use

  CircuitOperations(..., use_repetition_ids=True)
  warnings.warn(msg, FutureWarning)
/Users/nachoblancasrodriguez/anaconda3/envs/yquantum-quera/lib/python3.13/site-packages/cirq/circuits/circuit_operation.

00 540
01 463
11 494
10 503
